In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/covid-pneumonia-normal-chest-xray-images/PNEUMONIA/PNEUMONIA_1563.png
/kaggle/input/covid-pneumonia-normal-chest-xray-images/PNEUMONIA/PNEUMONIA_854.png
/kaggle/input/covid-pneumonia-normal-chest-xray-images/PNEUMONIA/PNEUMONIA_1404.png
/kaggle/input/covid-pneumonia-normal-chest-xray-images/PNEUMONIA/PNEUMONIA_1194.png
/kaggle/input/covid-pneumonia-normal-chest-xray-images/PNEUMONIA/PNEUMONIA_1772.png
/kaggle/input/covid-pneumonia-normal-chest-xray-images/PNEUMONIA/PNEUMONIA_934.png
/kaggle/input/covid-pneumonia-normal-chest-xray-images/PNEUMONIA/PNEUMONIA_1461.png
/kaggle/input/covid-pneumonia-normal-chest-xray-images/PNEUMONIA/PNEUMONIA_60.png
/kaggle/input/covid-pneumonia-normal-chest-xray-images/PNEUMONIA/PNEUMONIA_296.png
/kaggle/input/covid-pneumonia-normal-chest-xray-images/PNEUMONIA/PNEUMONIA_1011.png
/kaggle/input/covid-pneumonia-normal-chest-xray-images/PNEUMONIA/PNEUMONIA_1254.png
/kaggle/input/covid-pneumonia-normal-chest-xray-images/PNEUMONIA/PNEUMONIA_798.pn

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.metrics import top_k_categorical_accuracy

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import os
from glob import glob
from PIL import Image
import numpy as np
import pandas as pd
import cv2
import random
import albumentations as A
import keras
import matplotlib.cm as cm
import plotly.express as px
import plotly.figure_factory as ffis 
import matplotlib.pyplot as plt
import seaborn as sns
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
from keras.preprocessing import image
import tensorflow as tf
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, recall_score, accuracy_score, precision_score, f1_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.layers import Input, Conv2D, UpSampling2D, Concatenate, GlobalAveragePooling2D, Dense, Multiply, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [5]:
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow

In [6]:
datagen = ImageDataGenerator(
    rescale=1.0/255,
    samplewise_center=True,
    samplewise_std_normalization=True,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    cval=0,
    validation_split=0.2
)

In [7]:
train_dir = "/kaggle/input/covid-pneumonia-normal-chest-xray-images"
train_data = datagen.flow_from_directory(
    train_dir,
    subset="training",
    target_size=(224, 224),
    batch_size=64,
    class_mode="categorical",
    color_mode = "rgb",
    shuffle = True,
    seed = 2022)

Found 4183 images belonging to 3 classes.


In [8]:
validation_generator = datagen.flow_from_directory(
    train_dir,  # Use the correct path here
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    shuffle = False,
    subset='validation'
)

Found 1045 images belonging to 3 classes.


In [9]:
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=20,          # Number of epochs with no improvement to wait
    restore_best_weights=True,  # Restore the best model weights when stopping

)

In [10]:
from tensorflow.keras.applications import VGG19

In [11]:
num_classes = 3

In [12]:
base_model = VGG19(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)  # Replace 'num_classes' with the number of classes in your dataset

model = Model(inputs=base_model.input, outputs=predictions)

80134624/80134624 [==============================] - 0s 0us/step


In [13]:
for layer in base_model.layers:
    layer.trainable = False

In [14]:
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(train_data, epochs=10, validation_data=validation_generator, callbacks=[early_stopping])

Epoch 1/10
66/66 [==============================] - 103s 1s/step - loss: 0.4965 - accuracy: 0.7918 - val_loss: 423.4484 - val_accuracy: 0.8900
Epoch 2/10
66/66 [==============================] - 76s 1s/step - loss: 0.2778 - accuracy: 0.9041 - val_loss: 230.4540 - val_accuracy: 0.9100
Epoch 3/10
66/66 [==============================] - 75s 1s/step - loss: 0.2409 - accuracy: 0.9173 - val_loss: 615.1510 - val_accuracy: 0.8660
Epoch 4/10
66/66 [==============================] - 76s 1s/step - loss: 0.2427 - accuracy: 0.9115 - val_loss: 336.8652 - val_accuracy: 0.9110
Epoch 5/10
66/66 [==============================] - 75s 1s/step - loss: 0.2200 - accuracy: 0.9194 - val_loss: 1296.7786 - val_accuracy: 0.9110
Epoch 6/10
 1/66 [..............................] - ETA: 1:04 - loss: 0.1457 - accuracy: 0.9531